In [1]:
#Question 1.2
import quandl as ql
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [ ]:
api_key = 'NMrrPr8sm_tAvv6Uof9M'
apiurl = f'https://demo.quandl.com/api/v3/datasets/OPEC/ORB.csv?api_key={api_key}'
ETH = requests.get(apiurl).json()['Data']['Data']
Yt_1 = pd.DataFrame(ETH)[['time', 'open']].set_index('time')
ToTimeStamp = ETH[0]['time']
apiurl = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=2000&toTs={ToTimeStamp}&api_key={api_key}'
ETH1 = requests.get(apiurl).json()['Data']['Data']
Yt_2 = pd.DataFrame(ETH1)[['time', 'open']].set_index('time')
ToTimeStamp = ETH1[0]['time']
apiurl = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=2000&toTs={ToTimeStamp}&api_key={api_key}'
ETH2 = requests.get(apiurl).json()['Data']['Data']
Yt_3 = pd.DataFrame(ETH2)[['time', 'open']].set_index('time')
Yt_data = pd.concat([Yt_3,Yt_2,Yt_1])
Yt_data.index = pd.to_datetime(Yt_data.index, unit = 's')
Yt_data = Yt_data[ Yt_data.index > '2010-07-17'] 
Yt_data.reset_index(inplace=True)
Yt_data.rename(columns ={'time':'Date'}, inplace='True')
Yt_data.dtypes
ql.ApiConfig.api_key = "NMrrPr8sm_tAvv6Uof9M"
Xt = pd.DataFrame(ql.get("MULTPL/SP500_REAL_PRICE_MONTH"))
Xt = Xt[ Xt.index > '2010-07-17'] 
Xt.reset_index(inplace=True)
Xt.dtypes

In [ ]:
df = Xt.merge(Yt_data, how ='left', on='Date')
df.rename(columns ={'Value':'Xt','open':'Yt'}, inplace='True')
df

In [ ]:
plt.plot(df.Xt,df.Yt,'go')

In [ ]:
beta = ((np.multiply(df.Yt, df.Xt)).mean()-np.mean(df.Xt)*np.mean(df.Yt))/((np.multiply(df.Xt,df.Xt)).mean()-np.mean(df.Xt)*np.mean(df.Xt))
alpha =np.mean(df.Yt)-beta*np.mean(df.Xt)
print("alpha =",alpha, "\nbeta =",beta)

In [ ]:
type((np.multiply(df.Yt, df.Xt)))
y_hat = alpha + np.multiply(beta,df.Xt)
L = np.sum(np.multiply(df.Yt - y_hat,df.Yt - y_hat))
L

In [ ]:
result = sm.ols('Yt~Xt',data=df).fit()
print(result.summary())

In [ ]:
def  cal_cost(a,b,x,y):
        n = len(y)
        pred = a + np.multiply(b,x)
        cost = (1/2*n) * np.sum(np.square(pred-y))
        return cost

In [ ]:
itr = 1000
alpha = 2000
n = len(df.Yt)

loss_his = np.zeros((itr,1))
alpha_his = np.zeros((itr,1))

for i in range(itr):
        pred = alpha + np.multiply(beta,df.Xt)
        res= df.Yt-pred
        res_sum = sum(res)
        gradient = -(1/n)*0.1*res_sum
 
        alpha = alpha - gradient
    
        loss_his[i] = cal_cost(alpha,beta,df.Xt,df.Yt)
        alpha_his[i] = alpha
    
plt.plot(alpha_his,loss_his,'go')